In [ ]:
#impor packages
import pandas as pd
import numpy as np
import os
import pickle
import json
import re

In [ ]:
# location of raw data files
input_path = "/home/jupyter/datasets/raw/"

In [ ]:
#read LABEVENTS.csv file
df = pd.read_csv(filepath_or_buffer=input_path+'MICROBIOLOGYEVENTS.csv',dtype = {'SUBJECT_ID':"object", 
                                                                                 'HADM_ID':"object", 
                                                                                 'CHARTDATE':"object", 
                                                                                 'CHARTTIME':"object", 
                                                                                 'SPEC_ITEMID':"object",
                                                                                 'SPEC_TYPE_DESC':"object", 
                                                                                 'ORG_ITEMID':"object", 
                                                                                 'ORG_NAME':"object", 
                                                                                 'ISOLATE_NUM':"object", 
                                                                                 'AB_ITEMID':"object",
                                                                                 'AB_NAME':"object", 
                                                                                 'DILUTION_TEXT':"object", 
                                                                                 'DILUTION_COMPARISON':"object", 
                                                                                 'DILUTION_VALUE':"object",
                                                                                 'INTERPRETATION':"object"})

In [ ]:
df.head()

In [ ]:
#Dropping Row_ID 
df = df.drop(labels='ROW_ID',axis = 1)

In [ ]:
# Print datatype and non-null data points for each column
df.info(verbose=True,null_counts=True)

In [ ]:
# Replacing missing values in ORG_NAME with "Negative Culture"
df['ORG_NAME'] = df['ORG_NAME'].fillna('Negative Culture')

In [ ]:
df.head()

In [ ]:
# Dropping ITEM id columns as the text description of these columns is available
df = df.drop(columns=['CHARTDATE','SPEC_ITEMID','ORG_ITEMID','AB_ITEMID'])

In [ ]:
df.head()

In [ ]:
# Reomving Special Charecters from SPEC_TYPE_DESC and ORG_NAME
df['SPEC_TYPE_DESC'] = df['SPEC_TYPE_DESC'].map(lambda x : re.sub('[!"#$%&()*+,;<=>?@[\]^_`{|}~\n]', '', x)) 
df['ORG_NAME'] = df['ORG_NAME'].map(lambda x : re.sub('[!"#$%&()*+,;<=>?@[\]^_`{|}~\n]', '', x)) 

In [ ]:
# combine Anitbiotic Name and its dilution value
df['AB_NAME_DILUTION'] = df['AB_NAME']+'-'+df['DILUTION_TEXT']

In [ ]:
# Droping columns which are not required as their data exists in columns being retained
df = df.drop(labels = ['ISOLATE_NUM','AB_NAME','DILUTION_TEXT','DILUTION_COMPARISON','DILUTION_VALUE'], axis = 'columns')

In [ ]:
# Fill missing values with 'NaN'
df = df.fillna('NaN')

In [ ]:
df.head(100)

In [ ]:
# Combine all columns into a single column
df['microbioevents'] = df['SPEC_TYPE_DESC']+' '+df['ORG_NAME']+' '+df['AB_NAME_DILUTION']+' '+df['INTERPRETATION']

In [ ]:
# Drop columns with redundant info
df = df.drop(labels=['SPEC_TYPE_DESC','ORG_NAME','AB_NAME_DILUTION','INTERPRETATION'], axis = 'columns')

In [ ]:
# Drop rows with missing Charttime 
df = df[df['CHARTTIME']!='NaN'].reset_index(drop = True)

In [ ]:
df.head(100)

In [ ]:
# Split the each event by whitespace
df['microbioevents'] = df['microbioevents'].map(lambda x: x.split())

In [ ]:
# remove 'NaN' from all events
df['microbioevents'] = df['microbioevents'].map(lambda x: list(filter(lambda a: a!='NaN',x)))

In [ ]:
df.head(100)

In [ ]:
# create output path
mypath_output = "/home/jupyter/datasets/microbioevents/tokenized/"
import os
os.makedirs(mypath_output, exist_ok=True)

In [ ]:
# export the dataframe to JSON format
df.to_json(mypath_output+"microbioevents.json",orient = 'records') 